### Importing the Core Modules

In [100]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as opt
from torch.utils.data import random_split, DataLoader, TensorDataset
import numpy as np
import pandas as pd

### Taking a look at the Dataset

In [101]:
np_dataset = np.genfromtxt("winequality-red.csv", delimiter=';')
print(np_dataset[:3])

[[    nan     nan     nan     nan     nan     nan     nan     nan     nan
      nan     nan     nan]
 [ 7.4     0.7     0.      1.9     0.076  11.     34.      0.9978  3.51
   0.56    9.4     5.    ]
 [ 7.8     0.88    0.      2.6     0.098  25.     67.      0.9968  3.2
   0.68    9.8     5.    ]]


### Creating the input and output tensors

In [102]:
inputs = []
output = []

for record in np_dataset[1:]:
    inputs.append(record[:-1])
    output.append(record[-1])

inputs = torch.tensor(np.array(inputs).astype(np.float32))
output = torch.tensor(np.array(output).astype(np.float32))

### Initializing the Hyperparameters

In [103]:
batch_size = 32
train_size = 1_179
valid_size = 400
test_size = 20

input_size = 11
output_size = 1

h1_size = 32
h2_size = 64
h3_size = 128
h4_size = 256
h5_size = 200
h6_size = 150
h7_size = 100
h8_size = 50
h9_size = 10

### Creating the Dataloaders

In [104]:
train_ds, valid_ds, test_ds = random_split(TensorDataset(inputs, output), [train_size, valid_size, test_size])

train_dl = DataLoader(train_ds, batch_size, num_workers=2, pin_memory=True, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size, num_workers=2, pin_memory=True)

### Creating the class that will put all the data loaders on GPU

In [105]:
class DeviceLoader:
    def __init__(self, device, loader):
        self.device = device
        self.loader = loader

    def __to_device(self, data):
        if isinstance(data, (list, tuple)):
            on_device = [d.to(self.device, non_blocking=True) for d in data]
            return on_device
        return data.to(self.device, non_blocking=True)

    def __iter__(self):
        for batch in self.loader:
            yield self.__to_device(batch)

### Creating the Model

In [106]:
class WineModel(nn.Module):
    def __init__(self, in_size, h1_size, h2_size, h3_size, h4_size, h5_size, h6_size, h7_size, h8_size, h9_size, out_size):
        super().__init__()
        self.linear1 = nn.Linear(in_size, h1_size)
        self.linear2 = nn.Linear(h1_size, h2_size)
        self.linear3 = nn.Linear(h2_size, h3_size)
        self.linear4 = nn.Linear(h3_size, h4_size)
        self.linear5 = nn.Linear(h4_size, h5_size)
        self.linear6 = nn.Linear(h5_size, h6_size)
        self.linear7 = nn.Linear(h6_size, h7_size)
        self.linear8 = nn.Linear(h7_size, h8_size)
        self.linear9 = nn.Linear(h8_size, h9_size)
        self.linear10 = nn.Linear(h9_size, out_size)

    def __call__(self, input_batch):
        out1 = self.linear1(input_batch)
        active_out1 = F.relu(out1)
        out2 = self.linear2(active_out1)
        active_out2 = F.leaky_relu(out2)
        out3 = self.linear3(active_out2)
        active_out3 = F.sigmoid(out3)
        out4 = self.linear4(active_out3)
        active_out4 = F.leaky_relu(out4)
        out5 = self.linear5(active_out4)
        active_out5 = F.sigmoid(out5)
        out6 = self.linear6(active_out5)
        active_out6 = F.leaky_relu(out6)
        out7 = self.linear7(active_out6)
        active_out7 = F.sigmoid(out7)
        out8 = self.linear8(active_out7)
        active_out8 = F.leaky_relu(out8)
        out9 = self.linear9(active_out8)
        active_out9 = F.relu(out9)
        model_pred = self.linear10(active_out9)
        return model_pred

    def get_loss(self, batch):
        inputs_batch, output_batch = batch
        model_pred = self(inputs_batch)
        loss = F.mse_loss(model_pred, output_batch)
        return loss

    def __get_acc(self, batch):
        inputs_batch, output_batch = batch
        model_pred = self(inputs_batch)
        acc = 100 - (torch.mean(torch.abs((output_batch - model_pred) / output_batch)) * 100)
        return acc

    def __validation_step(self, valid_batch):
        loss = self.get_loss(valid_batch)
        acc = self.__get_acc(valid_batch)
        return {"valid_batch_loss": torch.sqrt(loss).item(), "valid_batch_acc": acc}

    def __validation_end(self, results):
        avg_loss = torch.tensor([b["valid_batch_loss"] for b in results]).mean().item()
        avg_acc = torch.tensor([b["valid_batch_acc"] for b in results]).mean().item()
        return {"valid_loss": avg_loss, "valid_acc": avg_acc}

    def evaluate(self, validation_loader):
        batch_results = [self.__validation_step(b) for b in validation_loader]
        return self.__validation_end(batch_results)

    def epoch_end(self, epoch, valid_results):
        return {"Epoch": epoch+1, "Loss": valid_results["valid_loss"], "Acc": valid_results["valid_acc"]}

    def predict(self, test_rec):
        inputs_test, output_test = test_rec
        model_pred = self(inputs_test.unsqueeze(dim=0))
        loss = F.mse_loss(model_pred, output_test)
        acc = 100 - torch.abs((output_test - model_pred) / output_test).item() * 100
        return {"Loss": loss.sqrt().item(), "Acc(%)": acc, "Actual": output_test.item(), "Predicted": model_pred.item()}

### Initializing the Model and moving all tenosrs to GPU

In [107]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

model = WineModel(input_size, h1_size, h2_size, h3_size, h4_size, h5_size, h6_size, h7_size, h8_size, h9_size, output_size).to(device)

train_loader = DeviceLoader(device, train_dl)
valid_loader = DeviceLoader(device, valid_dl)
test_loader = DeviceLoader(device, test_ds)

### Creating the training loop

In [108]:
def fit(model, epochs, train_loader, valid_loader, opt_fn=opt.SGD, lr=1e-2):
    history = []
    optim = opt_fn(model.parameters(), lr=lr)
    for epoch in range(epochs):
        for training_batch in train_loader:
            loss = model.get_loss(training_batch)
            loss.backward()
            optim.step()
            optim.zero_grad()

        valid_results = model.evaluate(valid_loader)
        epoch_results = model.epoch_end(epoch, valid_results)
        history.append(epoch_results)
        print(epoch_results)

    return history

### Training the Model

In [109]:
history = fit(model, 10, train_loader, valid_loader)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: UserWarning: Using a target size (torch.Size([27])) that is different to the input size (torch.Size([27, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.


{'Epoch': 1, 'Loss': 0.8486420512199402, 'Acc': 86.40647888183594}


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.


{'Epoch': 2, 'Loss': 0.864513099193573, 'Acc': 86.17929077148438}
{'Epoch': 3, 'Loss': 0.8788720369338989, 'Acc': 86.02088165283203}
{'Epoch': 4, 'Loss': 0.8502781987190247, 'Acc': 87.3994140625}
{'Epoch': 5, 'Loss': 1.0327157974243164, 'Acc': 83.10515594482422}
{'Epoch': 6, 'Loss': 0.8362969756126404, 'Acc': 86.72574615478516}
{'Epoch': 7, 'Loss': 0.9378591775894165, 'Acc': 88.22893524169922}
{'Epoch': 8, 'Loss': 1.1026383638381958, 'Acc': 81.41567993164062}
{'Epoch': 9, 'Loss': 0.8690319061279297, 'Acc': 86.12621307373047}
{'Epoch': 10, 'Loss': 0.9260424971580505, 'Acc': 88.14601135253906}


### Predicting resutls

In [110]:
for t in test_loader:
    print(model.predict(t))

{'Loss': 0.8539590835571289, 'Acc(%)': 85.76734811067581, 'Actual': 6.0, 'Predicted': 5.146040916442871}
{'Loss': 0.8536815643310547, 'Acc(%)': 85.77197343111038, 'Actual': 6.0, 'Predicted': 5.146318435668945}
{'Loss': 0.8539900779724121, 'Acc(%)': 85.7668325304985, 'Actual': 6.0, 'Predicted': 5.146009922027588}
{'Loss': 0.8535547256469727, 'Acc(%)': 85.77408790588379, 'Actual': 6.0, 'Predicted': 5.146445274353027}
{'Loss': 0.1462407112121582, 'Acc(%)': 97.07518573850393, 'Actual': 5.0, 'Predicted': 5.146240711212158}
{'Loss': 0.14629077911376953, 'Acc(%)': 97.0741843804717, 'Actual': 5.0, 'Predicted': 5.1462907791137695}
{'Loss': 0.8536901473999023, 'Acc(%)': 85.77183037996292, 'Actual': 6.0, 'Predicted': 5.146309852600098}
{'Loss': 0.14607715606689453, 'Acc(%)': 97.07845691591501, 'Actual': 5.0, 'Predicted': 5.1460771560668945}
{'Loss': 0.853508472442627, 'Acc(%)': 85.77485829591751, 'Actual': 6.0, 'Predicted': 5.146491527557373}
{'Loss': 0.8534760475158691, 'Acc(%)': 85.775399208068

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:69: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
